## Импорт библиотек

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## Загрузка и изучение данных

In [4]:
try:
    df = pd.read_csv('C:/Users/trombee/YandexDisk/Учеба/Я. Практикум/insurance (1).csv')
except:
    df = pd.read_csv('/datasets/insurance.csv')

In [5]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [61]:
df.info()

In [62]:
df.describe()

In [63]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [64]:
features.head()

In [65]:
target.head()

In [66]:
df['Страховые выплаты'].unique()

Были загружены данные по страховым выплатам клиентов. Данные чистые, без пропусков в данных. Используются типы данных int и float.\
В данных есть признаки и целевой признак, в виде страховых выплат для клиента. Которые в дальнейшем нужно будет разделить, соответствующим образом.\
Также это задача регрессии, т.к. в целевом признаке указано количество выплат по клиенту\
Данные распределены примерно поровну среди мужчин и женщин. Средний возраст клиентов 30 лет.

## Обоснование решения

**Ответ:** Качество линейной регрессии не изменится.

**Обоснование:** 

Предсказания на исходных данных будут равны $a = Xw$

В это уравнение необходимо подставить формулу обучения $w = (X^T X)^{-1} X^T y$

После подстановки получаем следующее выражение $a = X (X^T X)^{-1} X^T y$

Далее раскрываем выражение, используя свойства матриц $a = X X^{-1} (X^T)^{-1} X^T y$

По свойству единичной матрицы $ X X^{-1} = E $ получаем $ a = E (X^T)^{-1} X^T y $

Также по свойству единичной матрицы $ (X^T)^{-1} X^T = E $ получаем $ a = E E y $

Предсказания на зашифрованных данных будут равны $ a\scriptscriptstyle 1 \textstyle = XPw \scriptscriptstyle 1 $

Подставим формулу обучения $ w \scriptscriptstyle 1 \textstyle = ((X P)^T (X P))^{-1} (X P)^T y $

Получаем $ a \scriptscriptstyle 1 \textstyle = (X P) ((X P)^T (X P))^{-1} (X P)^T y $

Далее воспользуемся свойством обратной матрицы: $ ((X P)^T (X P))^{-1} = (X P)^{-1} ((X P)^T)^{-1} $

Получим из выражения $ a \scriptscriptstyle 1 \textstyle = (X P) (X P)^{-1} ((X P)^T)^{-1} (X P)^T y $ 

По свойствам единичных матриц, которые были использованы ранее, получим:

$$ a\scriptscriptstyle 1 \textstyle = E E y\scriptscriptstyle 1 $$

Получается, что $ a = E E y $ и $ a\scriptscriptstyle 1 \textstyle = E E y\scriptscriptstyle 1 $
Следовательно: $ a = a\scriptstyle 1 $

Получается нам нужно доказать следующее $ a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w' $

$ w = (X^T X)^{-1} X^T y $

$ w' = ((XP)^T XP)^{-1} (XP)^T y $

$ w' = (P^T (X^T X) P)^{-1} (XP)^T y $

Далее, по свойству обратимости $ (AB)^{-1} = B^{-1} A^{-1} $

Получим $ w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y $

По свойству $ A A^{-1} = A^{-1} A = E $, будет $ w' = P^{-1} (X^T X)^{-1} E X^T y $

Согласно свойства $ AE = EA = A $ и выражения $ w = (X^T X)^{-1} X^T y $

Будет $ w' = P^{-1} w $

Тем самым доказано, что $ a = a\scriptstyle 1 $

### 

Исходя из доказательства выше, можно сказать, что качество линейной регрессии не изменится, если умножить признаки на обратную матрицу\
Поэтому, можно использовать этот алгоритм для преобразования данных

Тогда алгоритм будет выглядеть следующим образом:
 - Разделим признаки и целевой признак на обучающую и тестовую выборку;
 - Далее обучим модель и проверим метрику r2_score на исходных данных
 - Далее создадим обратную матрицу;
 - Умножим обратную матрицу на матрицу признаков;
 - Далее обучение модели на преобразованных данных и проверка метрика r2_score на этих данных;
 - Сравнение двух метрик
 
Выбран данный алгоритм, т.к. в итоге метрики r2_score при обучении на исходных данных, и на преобразованных практически не отличаются. А при этом, данные будут защищены.

### 

In [67]:
features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                            test_size=0.25,
                                                                            random_state=12345)

In [68]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(f'Метрика R2_score ={r2_score(target_test, predictions)}')

In [69]:
matrix = np.random.randn(4, 4)
inv_matrix = np.linalg.inv(matrix)

In [70]:
features_inv = features @ matrix

In [71]:
model_inv = LinearRegression()
model_inv.fit(features_inv, target)
predictions_inv = model_inv.predict(features_inv)
print(f'Метрика R2_score на преобразованных данных = {r2_score(target, predictions_inv)}')

По итогу проверки данного алгоритму получилось, что метрика r2_score показывает одинаковый результат на исходных данных и на преобразованных данных.

<div style="border:solid orange 2px; padding: 20px">

## Вывод

В ходе работы на проектом "защита персональных данных клиента" был загружен и изучен набор данных о страховых выплатах клиентов.

Были сформированы выборки из признаков и целевого признака.

Также было доказано, что данные можно зашифровать путем умножения исходных признаков на обратную матрицу. После чего качество модели линейной регресси не изменилось, а данные остались защищены от злоумышленников.